Import libraries

In [1]:
from bs4 import BeautifulSoup
import os
import datetime
import time
import csv
import sys
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd

Crawl list of companies

In [7]:
DEBUG = 1

def connectchrome():
    options = Options()
    # if you want to hide browser window, uncomment line below
    #options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(executable_path= r'C:\\Users\\user\\Desktop\\python\\banyous\\1-Quora-scrapping\\chromedriver.exe', options = options)
    driver.maximize_window()
    time.sleep(2)
    return driver

driver = connectchrome()
driver.implicitly_wait(3)

# Log-in
driver.get('https://www.businessmodelideas.com/login')
login_button = driver.find_element_by_xpath("//div[@class='col-12 text-lg-right text-left']/button[@type='submit' and @class='cta']")

driver.find_element_by_name('login').send_keys('nrdk@ds.seoultech.ac.kr')
driver.find_element_by_name('password').send_keys('qkaqlcRhc13!')
login_button.click()
driver.implicitly_wait(3)

company_list = []
url1 = "https://www.businessmodelideas.com/explore"

driver.get(url1)
driver.implicitly_wait(3)

# Find company titles (h2 tags) and Save
company_elements = driver.find_elements_by_class_name("company-title")
company_names = [element.text for element in company_elements]

with open('company_list.txt', 'w', encoding='utf-8') as file:
    for name in company_names:
        file.write(name + "\n")

Collecting enterprise-specific data

In [34]:
DEBUG = 1

def connectchrome():
    options = Options()
    # if you want to hide browser window, uncomment line below
    #options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(executable_path= r'C:\\Users\\user\\Desktop\\python\\banyous\\1-Quora-scrapping\\chromedriver.exe', options = options)
    driver.maximize_window()
    time.sleep(2)
    return driver

driver = connectchrome()
driver.implicitly_wait(3)

# Log-in
driver.get('https://www.businessmodelideas.com/login')
login_button = driver.find_element_by_xpath("//div[@class='col-12 text-lg-right text-left']/button[@type='submit' and @class='cta']")

driver.find_element_by_name('login').send_keys('nrdk@ds.seoultech.ac.kr')
driver.find_element_by_name('password').send_keys('qkaqlcRhc13!')
login_button.click()
driver.implicitly_wait(3)


############################# Pilot
# Amazon-retail test
driver.get('https://www.businessmodelideas.com/company/amazon-retail')
driver.implicitly_wait(3)

# Company name
#company_element = driver.title
#if "|" in page_title:
#    company_name = page_title.split("|")[0].strip()
#else:
#    company_name = page_title

# Description paragraph
paragraph_element = driver.find_element_by_xpath("//div[@class='description']/p")
paragraph_text = paragraph_element.text

### Model Canvas
# Column headers
headers = [
    "Key Partners", "Key Activities", "Key Resources", "Value Propositions", 
    "Customer Relationships", "Channels", "Customer Segments", 
    "Cost Structure", "Revenue Streams"
]


data = {"Description": [paragraph_text]}  # Description


for header in headers:
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, f"//div[@class='cube-heading mb-2' and contains(text(), '{header}')]"))
        )
    except TimeoutException:
        print(f"Timeout waiting for header: {header}")
        continue

    # 헤더에 해당하는 cube-element 내부의 span 텍스트들을 추출
    span_elements = driver.find_elements_by_xpath(f"//div[@class='cube-heading mb-2' and contains(text(), '{header}')]/following-sibling::div[contains(@class, 'cube-element')]/span")
    span_texts = [element.text for element in span_elements]

    data[header] = span_texts

# DataFrame generation
df = pd.DataFrame(dict([(k, pd.Series(v, dtype='object')) for k,v in data.items()]))

# Save to .csv file
df.to_csv("collected_data.csv", index=False)

# Close the Webdriver
driver.quit()

In [ ]:
def save_data(data, mode='a'):
    with open("business_ideas.csv", mode, newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        if mode == 'w':
            writer.writerow(["company", "Description", "Employees", "Key partners", "Key Activities", "Key Resources","Value Propositions", "Customer Relationships", "Channels", "Customer Segements", "Cost structure", "Revenue Streams"])  # Header
        writer.writerows(data)
        

In [ ]:
#company_element = driver.title

#if "|" in page_title:
#    company_name = page_title.split("|")[0].strip()
#else:
#    company_name = page_title

## Model Canvas
classes = [
    "boxed-content.cube.big-cube-left", # Key partners
    "boxed-content.cube.small.small-cube-top", # Key Activities
    "boxed-content.cube.cube.small-cube-bottom", # Key Resources
    "boxed-content.cube.big-cube-middle", # Value Propositions
    "boxed-content.cube.cube-small.small-cube-top", # Customer Relationships
    "boxed-content.cube.cube-small.small-cube-bottom", # Channels
    "boxed-content.cube.big-cube-right", # Customer Segments
    "boxed-content.wide-cube-left", # Cost Structure
    "boxed-content.wide-cube-right" # Revenue Streams
]